In [2]:
import os
import re
import pandas as pd

In [3]:
import os
import joblib

from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer

# scoring in anything
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import f1_score, matthews_corrcoef
from sklearn.metrics import precision_score, recall_score

In [4]:
df = pd.read_csv("creditcard.csv")
# Determine number of fraud cases in dataset
fraud = df[df['Class'] == 1]
valid = df[df['Class'] == 0]
outlierFraction = len(fraud) / float(len(valid))
print(outlierFraction)
print('Fraud Cases: {}'.format(len(df[df['Class'] == 1])))
print('Valid Transactions: {}'.format(len(df[df['Class'] == 0])))
# dividing the X and the Y from the dataset
X = df.drop(['Class'], axis=1)
Y = df["Class"]
print(X.shape)
print(Y.shape)
# getting just the values for the sake of processing
# (its a numpy array with no columns)
xData = X.values
yData = Y.values

0.0017304750013189597
Fraud Cases: 492
Valid Transactions: 284315
(284807, 30)
(284807,)


In [5]:
fraud

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [6]:
fraud.to_json(orient='records')

'[{"Time":406.0,"V1":-2.3122265423,"V2":1.9519920106,"V3":-1.6098507323,"V4":3.9979055875,"V5":-0.5221878647,"V6":-1.4265453192,"V7":-2.5373873062,"V8":1.3916572483,"V9":-2.7700892772,"V10":-2.7722721447,"V11":3.2020332071,"V12":-2.8999073885,"V13":-0.5952218813,"V14":-4.2892537824,"V15":0.3897241203,"V16":-1.1407471798,"V17":-2.8300556745,"V18":-0.0168224682,"V19":0.416955705,"V20":0.1269105591,"V21":0.5172323709,"V22":-0.0350493686,"V23":-0.4652110762,"V24":0.3201981985,"V25":0.0445191675,"V26":0.1778397983,"V27":0.2611450026,"V28":-0.1432758747,"Amount":0.0,"Class":1},{"Time":472.0,"V1":-3.043540624,"V2":-3.1573071209,"V3":1.08846278,"V4":2.2886436184,"V5":1.3598051297,"V6":-1.064822523,"V7":0.3255742662,"V8":-0.0677936532,"V9":-0.2709528362,"V10":-0.8385865646,"V11":-0.4145754483,"V12":-0.5031408596,"V13":0.6765015446,"V14":-1.6920289331,"V15":2.0006348391,"V16":0.6667796959,"V17":0.5997174138,"V18":1.7253210075,"V19":0.2833448301,"V20":2.1023387926,"V21":0.6616959248,"V22":0.43547

In [7]:
valid

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [8]:
type(xData)

numpy.ndarray

In [9]:
xData[0]

array([ 0.00000000e+00, -1.35980713e+00, -7.27811733e-02,  2.53634674e+00,
        1.37815522e+00, -3.38320770e-01,  4.62387778e-01,  2.39598554e-01,
        9.86979013e-02,  3.63786970e-01,  9.07941720e-02, -5.51599533e-01,
       -6.17800856e-01, -9.91389847e-01, -3.11169354e-01,  1.46817697e+00,
       -4.70400525e-01,  2.07971242e-01,  2.57905802e-02,  4.03992960e-01,
        2.51412098e-01, -1.83067779e-02,  2.77837576e-01, -1.10473910e-01,
        6.69280749e-02,  1.28539358e-01, -1.89114844e-01,  1.33558377e-01,
       -2.10530535e-02,  1.49620000e+02])

In [10]:
yData[0]

0

In [12]:
# Building the Random Forest Classifier (RANDOM FOREST) , Logistic Regression , LinearSVC
clfs = [LogisticRegression()]#n_estimators=500 for random forst
best_f1 = 0
best_clf = None
# dividing the X and the Y from the dataset
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2)
for clf in clfs:
    print(f"training clf {clf}")
    clf = Pipeline([('clf', clf)])
    clf.fit(x_train, y_train)
    f1 = f1_score(y_test, clf.predict(x_test), average='micro')
    print(f"clf {clf} has f1 score of {f1}")
    if f1 > best_f1:
        best_f1 = f1
        best_clf = clf
print(f"trained with best f1 of {best_f1} \n\n")

training clf LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)


F:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


clf Pipeline(memory=None,
         steps=[('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='warn', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False) has f1 score of 0.9990344440153085
trained with best f1 of 0.9990344440153085 




In [13]:
type(x_test)

pandas.core.frame.DataFrame

In [14]:
x_test

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
110364,71793.0,1.450417,-1.008258,0.547631,-1.289949,-1.550380,-0.754580,-0.995029,-0.119841,-1.833490,...,-0.384697,-0.167281,-0.069526,0.106272,0.370237,0.261746,-0.196008,0.045002,0.020556,15.00
193372,130111.0,1.806698,-1.486776,-1.237177,-0.385168,-0.954744,-0.817266,-0.243522,-0.402629,-0.011034,...,0.402192,0.589308,1.648583,-0.343514,0.205893,0.440886,0.265941,-0.043339,-0.034206,209.00
220069,142008.0,1.905455,-0.919774,-1.555953,-0.502141,0.301364,0.863037,-0.420524,0.107457,-0.439966,...,-0.515242,0.079123,0.861439,0.070042,-0.311335,0.018588,0.038033,0.052978,-0.044267,79.00
214963,139883.0,-1.357257,0.180432,0.875156,-0.305253,0.576101,-0.026724,0.560755,-0.074989,0.777317,...,-0.132531,0.083016,0.708396,-0.660118,-0.598467,1.346812,0.828483,-0.273818,0.181160,57.10
84864,60496.0,-2.573192,2.218057,0.626278,-1.902088,0.372946,0.083876,-0.054230,-4.668976,1.579083,...,-0.197849,3.432388,-1.531803,0.219087,-0.501557,-0.074619,0.694149,-0.472745,-0.788122,0.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132570,80034.0,-0.345049,0.462800,1.039934,-1.196388,-0.215608,-1.463005,0.719640,-0.457525,-1.866428,...,0.133888,0.306338,0.841908,-0.168642,0.776850,0.341690,-0.157217,-0.081577,0.028747,15.00
116906,74474.0,-0.482158,0.928183,1.444300,-0.207997,0.191323,-0.400719,0.603384,0.090392,-0.666410,...,0.164743,-0.186100,-0.527389,-0.048439,-0.010909,-0.167102,0.073058,0.257848,0.109622,10.99
274983,166307.0,1.957705,0.022704,-1.875959,0.321851,0.441883,-0.699295,0.105225,-0.139906,0.088707,...,-0.061566,0.233383,0.672755,0.024544,0.668312,0.029383,0.626016,-0.073563,-0.036323,38.49
87823,61860.0,-0.990432,0.213828,2.326756,-0.102401,0.135111,0.525852,0.277117,0.205178,0.135755,...,-0.160369,0.189891,0.633211,-0.037435,-0.351006,-0.164556,-0.524501,-0.125552,-0.112269,49.83


In [15]:
# Evaluating the classifier 
# printing every score of the classifier 
# scoring in anything 
from sklearn.metrics import classification_report, accuracy_score  
from sklearn.metrics import precision_score, recall_score 
from sklearn.metrics import f1_score, matthews_corrcoef 
from sklearn.metrics import confusion_matrix 
  
n_outliers = len(fraud) 
n_errors = (yPred != yTest).sum() 
print("The model used is Random Forest classifier") 
  
acc = accuracy_score(yTest, yPred) 
print("The accuracy is {}".format(acc)) 
  
prec = precision_score(yTest, yPred) 
print("The precision is {}".format(prec)) 
  
rec = recall_score(yTest, yPred) 
print("The recall is {}".format(rec)) 
  
f1 = f1_score(yTest, yPred) 
print("The F1-Score is {}".format(f1)) 
  
MCC = matthews_corrcoef(yTest, yPred) 
print("The Matthews correlation coefficient is{}".format(MCC)) 

NameError: name 'yPred' is not defined

In [19]:
clf.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
print(clf.predict([x_test.iloc[0]])[0])

0


In [18]:
x_test.iloc[0].to_json(orient='index')

'{"Time":71793.0,"V1":1.4504168754,"V2":-1.008258455,"V3":0.5476313958,"V4":-1.2899492445,"V5":-1.5503799635,"V6":-0.7545802713,"V7":-0.9950285914,"V8":-0.1198408348,"V9":-1.8334900355,"V10":1.4031686884,"V11":-0.00662216,"V12":-0.8191470208,"V13":0.1679562213,"V14":-0.208649112,"V15":0.8225275173,"V16":-0.7775661523,"V17":0.9208826051,"V18":-0.5666365658,"V19":-0.7250835033,"V20":-0.3846973385,"V21":-0.1672810431,"V22":-0.0695263927,"V23":0.1062720096,"V24":0.370236826,"V25":0.2617462692,"V26":-0.1960084561,"V27":0.0450021458,"V28":0.0205562242,"Amount":15.0}'